In [8]:
from typing import Iterable, List, Dict, Optional, Tuple
import pandas as pd
import numpy as np
df = pd.read_excel(r'C:\Users\anama\Downloads\excel1Test.xls')

In [9]:
df

,acbm,s ar,cola,var,pos,pn,sn,c_acbm,c_var,c_pos,c_pn,c_sn,f inst,vidas
0,12,av,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-09,FH
1,12,av,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-09,FT
2,12,av,1,1,1,1,1,11.0,11.0,11.0,11-1-a,aqwe,2024-10-23,FH
3,12,av,1,1,1,1,1,11.0,11.0,12.0,12-1a,ggg,2024-05-14,FH
4,12,av,1,1,1,1,1,11.0,11.0,13.0,13-1a,sss,2025-01-02,FT
5,12,av,1,1,1,1,1,11.0,11.0,11.0,11-1-a,aqwe,2024-10-23,FT


In [10]:
def eliminar_columnas(df: pd.DataFrame, columnas: Iterable[str], errores: str = "ignore") -> pd.DataFrame:
    """
    errores: "ignore" no falla si no existe la columna; "raise" para forzar error.
    """
    df = df.copy()
    return df.drop(columns=list(columnas), errors=errores)

In [11]:
df_clean=eliminar_columnas(df,["vidas"])

In [12]:
df_clean

,acbm,s ar,cola,var,pos,pn,sn,c_acbm,c_var,c_pos,c_pn,c_sn,f inst
0,12,av,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-09
1,12,av,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-09
2,12,av,1,1,1,1,1,11.0,11.0,11.0,11-1-a,aqwe,2024-10-23
3,12,av,1,1,1,1,1,11.0,11.0,12.0,12-1a,ggg,2024-05-14
4,12,av,1,1,1,1,1,11.0,11.0,13.0,13-1a,sss,2025-01-02
5,12,av,1,1,1,1,1,11.0,11.0,11.0,11-1-a,aqwe,2024-10-23


In [13]:
df_sin_dupes = df_clean.drop_duplicates(keep="first")

In [14]:
df_sin_dupes

,acbm,s ar,cola,var,pos,pn,sn,c_acbm,c_var,c_pos,c_pn,c_sn,f inst
0,12,av,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-09
2,12,av,1,1,1,1,1,11.0,11.0,11.0,11-1-a,aqwe,2024-10-23
3,12,av,1,1,1,1,1,11.0,11.0,12.0,12-1a,ggg,2024-05-14
4,12,av,1,1,1,1,1,11.0,11.0,13.0,13-1a,sss,2025-01-02


In [17]:
def construir_columna(df: pd.DataFrame, columnas: Iterable[str], nombre_col: str, separador: str) -> pd.DataFrame:
    """
    Crea un Columna concatenando otras columnas como strings (tras trim). NaN -> "".
    """
    df = df.copy()
    cols = [c for c in columnas]
    piezas = []
    for c in cols:
        piezas.append(df[c].astype(str).fillna("").str.strip())
    df[nombre_col] = piezas[0]
    for p in piezas[1:]:
        df[nombre_col] = df[nombre_col] + separador + p
    return df

In [18]:
df_sin_dupes_con_ac = construir_columna(df_sin_dupes,columnas=["s ar","cola"],nombre_col="AC",separador="-")

In [19]:
df_sin_dupes_con_ac

,acbm,s ar,cola,var,pos,pn,sn,c_acbm,c_var,c_pos,c_pn,c_sn,f inst,AC
0,12,av,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-09,av-1
2,12,av,1,1,1,1,1,11.0,11.0,11.0,11-1-a,aqwe,2024-10-23,av-1
3,12,av,1,1,1,1,1,11.0,11.0,12.0,12-1a,ggg,2024-05-14,av-1
4,12,av,1,1,1,1,1,11.0,11.0,13.0,13-1a,sss,2025-01-02,av-1


In [21]:
def añade_col_condicional(
    df: pd.DataFrame,
    nombre_columna: str,
    condicion: str,
    valor_si: any,
    valor_no: any
) -> pd.DataFrame:
    """
    Añade una nueva columna con valores definidos por una condición.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame original.
    nombre_columna : str
        Nombre de la columna nueva.
    condicion : str
        Condición a evaluar sobre df (debe devolver una Serie booleana).
        Ejemplo: "df['f inst'].isna()"
    valor_si : any
        Valor asignado si la condición se cumple (True).
    valor_no : any
        Valor asignado si la condición no se cumple (False).

    Retorna:
    --------
    DataFrame con la nueva columna añadida.
    """
    df = df.copy()
    try:
        mask = eval(condicion, {"np": np, "pd": pd, "df": df})
        if not isinstance(mask, (pd.Series, np.ndarray, list)):
            raise ValueError("La condición debe devolver una serie booleana.")
        df[nombre_columna] = np.where(mask, valor_si, valor_no)
    except Exception as e:
        raise ValueError(f"Error al evaluar la condición: {e}")
    return df

In [22]:
df_sin_dupes_con_ac_conMDMD = añade_col_condicional(
    df_sin_dupes_con_ac,
    nombre_columna="es_hueco",
    condicion="df['f inst'].isna()",
    valor_si="D",   # D si está vacía
    valor_no="M"    # M si tiene fecha
)

In [23]:
df_sin_dupes_con_ac_conMDMD

,acbm,s ar,cola,var,pos,pn,sn,c_acbm,c_var,c_pos,c_pn,c_sn,f inst,AC,es_hueco
0,12,av,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-09,av-1,M
2,12,av,1,1,1,1,1,11.0,11.0,11.0,11-1-a,aqwe,2024-10-23,av-1,M
3,12,av,1,1,1,1,1,11.0,11.0,12.0,12-1a,ggg,2024-05-14,av-1,M
4,12,av,1,1,1,1,1,11.0,11.0,13.0,13-1a,sss,2025-01-02,av-1,M


In [45]:
tabla2_parte1 = df_sin_dupes_con_ac_conMDMD.iloc[:4].copy()
tabla2_parte1["es_hueco"] = ["D","M", "M", "D"]

tabla2_nuevas = pd.DataFrame({
    "acbm": [13, 14, 15],
    "s ar": ["bx", "cz", "dx"],
    "cola": [2, 3, 4],
    "var": [1, 2, 3],
    "pos": [2, 2, 3],
    "pn": [1, 2, 3],
    "sn": [1, 2, 3],
    "c_acbm": [21.0, 22.0, 23.0],
    "c_var": [21.0, 22.0, 23.0],
    "c_pos": [21.0, 22.0, 23.0],
    "c_pn": ["21-1a", "22-1a", "23-1a"],
    "c_sn": ["bbb", "ccc", "ddd"],
    "f inst": ["2025-03-01", "2025-04-01", "2025-05-01"],
    "AC": ["bx-1", "cz-1", "dx-1"],
    "es_hueco": ["M", "M", "D"]
})
tabla2 = pd.concat([tabla2_parte1, tabla2_nuevas], ignore_index=True)

In [46]:
def limpia_f_inst_si_hueco(df: pd.DataFrame, col_estado: str = "es_hueco", col_fecha: str = "f inst") -> pd.DataFrame:
    """
    Si el valor de col_estado == 'D', entonces col_fecha se deja vacío (NaN).
    """
    df = df.copy()
    if col_estado not in df.columns or col_fecha not in df.columns:
        raise ValueError(f"Las columnas '{col_estado}' y '{col_fecha}' deben existir en el DataFrame.")
    
    mask = df[col_estado].astype(str).str.upper() == "D"
    df.loc[mask, col_fecha] = np.nan
    return df

In [47]:
tabla2 = limpia_f_inst_si_hueco(tabla2)

In [48]:
tabla2

,acbm,s ar,cola,var,pos,pn,sn,c_acbm,c_var,c_pos,c_pn,c_sn,f inst,AC,es_hueco
0,12,av,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,av-1,D
1,12,av,1,1,1,1,1,11.0,11.0,11.0,11-1-a,aqwe,2024-10-23 00:00:00,av-1,M
2,12,av,1,1,1,1,1,11.0,11.0,12.0,12-1a,ggg,2024-05-14 00:00:00,av-1,M
3,12,av,1,1,1,1,1,11.0,11.0,13.0,13-1a,sss,NaN,av-1,D
4,13,bx,2,1,2,1,1,21.0,21.0,21.0,21-1a,bbb,2025-03-01,bx-1,M
5,14,cz,3,2,2,2,2,22.0,22.0,22.0,22-1a,ccc,2025-04-01,cz-1,M
6,15,dx,4,3,3,3,3,23.0,23.0,23.0,23-1a,ddd,NaN,dx-1,D


In [64]:
def comparar_tablas(
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    claves: list[str],
    nombre_tabla1: str,
    nombre_tabla2: str,
    nombre_columna_resultado: str = "resultado_comparacion"
) -> pd.DataFrame:
    """
    Compara dos DataFrames por columnas clave.
    Si todos los campos son iguales -> 'OK'
    Si hay alguna diferencia -> 'DISCREPANCIA'
    
    Retorna un DataFrame con los campos combinados y la columna de resultado.
    """
    # Copias limpias
    df1 = df1.copy()
    df2 = df2.copy()

    # Unir las dos tablas por las claves (inner = solo coincidencias)
    merged = df1.merge(df2, on=claves, how="outer", suffixes=("_1", "_2"), indicator=True)

    # Determinar columnas comparables (todas menos las claves y _merge)
    all_cols = [c.replace("_1", "") for c in merged.columns if c.endswith("_1")]
    comparables = [c for c in all_cols if c not in claves]

    # Crear columna de resultado
    def comparar_fila(row):
        if row["_merge"] == "left_only":
            return "REVISAR: No está en " + nombre_tabla2 + ", pero sí en "+ nombre_tabla1
        elif row["_merge"] == "right_only":
            return "REVISAR: No está en " + nombre_tabla1 + ", pero sí en "+ nombre_tabla2
        else:  # _merge == 'both'
            for col in comparables:
                val1 = row.get(f"{col}_1", np.nan)
                val2 = row.get(f"{col}_2", np.nan)
                if pd.isna(val1) and pd.isna(val2):
                    continue
                if str(val1) != str(val2):
                    return "DISCREPANCIA"
            return "OK"

    merged[nombre_columna_resultado] = merged.apply(comparar_fila, axis=1)

    # Opcional: quedarnos solo con columnas originales (de tabla1) + resultado
    resultado = merged[[c for c in merged.columns if c.endswith("_1") or c in claves or c == nombre_columna_resultado]]
    resultado.columns = [c.replace("_1", "") for c in resultado.columns]

    return resultado

In [31]:
claves = ["acbm", "s ar", "cola", "var", "pos", "pn", "sn", "c_acbm", "c_var", "c_pos", "c_pn", "c_sn", "AC"]

In [65]:
resultado = comparar_tablas(df_sin_dupes_con_ac_conMDMD, tabla2, claves, "RDCD", "HUECOS e INVENTARIO")

In [66]:
resultado

,acbm,s ar,cola,var,pos,pn,sn,c_acbm,c_var,c_pos,c_pn,c_sn,f inst,AC,es_hueco,resultado_comparacion
0,12,av,1,1,1,1,1,11.0,11.0,11.0,11-1-a,aqwe,2024-10-23,av-1,M,OK
1,12,av,1,1,1,1,1,11.0,11.0,12.0,12-1a,ggg,2024-05-14,av-1,M,OK
2,12,av,1,1,1,1,1,11.0,11.0,13.0,13-1a,sss,2025-01-02,av-1,M,DISCREPANCIA
3,12,av,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,2023-09-09,av-1,M,DISCREPANCIA
4,13,bx,2,1,2,1,1,21.0,21.0,21.0,21-1a,bbb,NaT,bx-1,NaN,"REVISAR: No está en RDCD, pero sí en HUECOS e ..."
5,14,cz,3,2,2,2,2,22.0,22.0,22.0,22-1a,ccc,NaT,cz-1,NaN,"REVISAR: No está en RDCD, pero sí en HUECOS e ..."
6,15,dx,4,3,3,3,3,23.0,23.0,23.0,23-1a,ddd,NaT,dx-1,NaN,"REVISAR: No está en RDCD, pero sí en HUECOS e ..."
